# Subproblem for KEP

In [ ]:
function kep_subproblem(π,node)
    SP=Model(optimizer_with_attributes(() -> Gurobi.Optimizer(GUROBI_ENV)))
    set_optimizer_attribute(SP, "OutputFlag", 0)
    
    @variable(SP,x[i in 1:ne(pool)],Bin) # edge variable
    @variable(SP,y[i in 1:nv(pool)],Bin) # node variable
    # flow constraint
    for i in 1:nv(pool)
        if length(inneighbors(pool,i))>=1
            @constraint(SP,sum(x[findall(a->a==Edge(k,i), e)] for k in inneighbors(pool,i))[1] == y[i])
        end
        if length(outneighbors(pool,i))>=1
            @constraint(SP, sum(x[findall(a->a==Edge(i,k), e)] for k in outneighbors(pool,i))[1] == y[i])
        end
    end
    # length constraint
    @constraint(SP,sum(x[i] for i in 1:ne(pool)) <= L)
    @objective(SP,Max,sum(weight[i]*x[i] for i=1:ne(pool)) - sum(π[i]*y[i] for i=1:nv(pool)))
    
    # branching constraint
    for v in tree[node].setzero
        @constraint(SP,y[v]<=0)
    end

    for v in tree[node].setone
        @constraint(SP,y[v]>=1)
    end
    

    optimize!(SP)

    # If the subproblem is solved to optimality then return its objective value
    # and the optimal pattern solution for the machine
    # Else the pricing problem is infeasible return Inf, []
    if JuMP.termination_status(SP)==MOI.OPTIMAL
        return JuMP.objective_value(SP),JuMP.value.(y),JuMP.value.(x)
    else
        return Inf,[]
    end
end